In [49]:
import pandas as pd
import urllib.parse as urlparse
import json
import re
import requests
bridge_file = open('./Bridge.txt','r')
key_list = bridge_file.readlines()
YT_API_KEY = key_list[0]
PERS_API_KEY = key_list[1]
bridge_file.close()

In [10]:
def countUrlType(input_dataset):
    channel_count = 0
    video_count = 0
    other_count = 0
    for idx in input_dataset.index:
        if input_dataset['url_type'][idx] == 'channel':
            channel_count=channel_count+1
        elif input_dataset['url_type'][idx] == 'video':
            video_count=video_count+1
        else:
            other_count=other_count+1
    return channel_count,video_count,other_count

In [12]:
def expanded_url_from(urllist_item):
    if 'expanded_url' in urllist_item:
        index_one = urllist_item.index('expanded_url')+16
        ext_url = ''
        for x in range(index_one,len(urllist_item)):
            if urllist_item[x] == '\'':
                return ext_url
            else:
                ext_url = ext_url+urllist_item[x]
        return urllist_item[index_one:index_two]
    else:
        return urllist_item
#expanded_url_from("[{'url': 'https://t.co/cZdjZpdrGh', 'expanded_url': 'https://youtu.be/ymdrOPAzLqc', 'display_url': 'youtu.be/ymdrOPAzLqc', 'indices': [169, 192]}]")

In [13]:
def checkSuccess(ch_count, vi_count, ot_count, sup_total):
    if (ch_count+vi_count+ot_count) == sup_total:
        return True
    else:
        return False

In [14]:
def getIDfromURL(yt_url):
    link_type = ''
    link_id = ''
    if '/c/' in yt_url or '/channel/' in yt_url:
        if '/c/' in yt_url:
            c_index = yt_url.index('/c/')
            if '?' in yt_url:
                q_index = yt_url.index('?')
                link_type = 'channel'
                link_id = yt_url[c_index+3:q_index]
            else:
                link_type = 'channel'
                link_id = yt_url[c_index+3:]
        elif '/channel/' in yt_url:
            c_index = yt_url.index('/channel/')
            if '?' in yt_url:
                q_index = yt_url.index('?')
                link_type = 'channel'
                link_id = yt_url[c_index+9:q_index]
            else:
                link_type = 'channel'
                link_id = yt_url[c_index+9:]
    elif '/watch?v=' in yt_url or '/v=' in yt_url or '/shorts/' in yt_url or '.be/' in yt_url:
        if '/watch?v=' in yt_url:
            w_index = yt_url.index('/watch?v=')
            link_type = 'video'
            link_id = yt_url[w_index+9:w_index+20]
        elif '/v=' in yt_url:
            w_index = yt_url.index('/v=')
            link_type = 'video'
            link_id = yt_url[w_index+3:w_index+14]
        elif '/shorts/' in yt_url:
            w_index = yt_url.index('/shorts/')
            link_type = 'video'
            link_id = yt_url[w_index+8:w_index+19]  
        elif '.be/' in yt_url:
            w_index = yt_url.index('.be/')
            link_type = 'video'
            link_id = yt_url[w_index+4:w_index+20]
    else:
        link_type = 'other'
        link_id = 'NA'
    return link_type,link_id

In [15]:
def takeDateQuery():
    month_dict = {'01': 'Jan','02': 'Feb','03': 'Mar','04': 'Apr',
             '05': 'May','06': 'Jun','07': 'Jul','08': 'Aug',
             '09': 'Sep','10': 'Oct','11': 'Nov','12': 'Dec'}
    date_query = input('Input date in MM/DD/YYYY format').split('/')
    month_query = month_dict[date_query[0]]
    day_query = date_query[1]
    year_query = date_query[2]
    print(month_query,day_query,year_query)
    return month_query,day_query,year_query

In [4]:
twitter_dataset_onlyurl = pd.read_csv('./datasets/yt_onlyurl.csv')

In [4]:
twitter_dataset_onlyurl.columns

Index(['tweetid', 'urls_list', 'url'], dtype='object')

In [5]:
twitter_dataset_onlyurl.size

3908106

In [5]:
twitter_dataset = pd.read_csv('./datasets/yturl_twitter.csv',lineterminator="\n",low_memory=False)

In [6]:
twitter_dataset

,tweetid,userid,screen_name,date,lang,location,description,place_id,place_url,place_type,...,state,country,rt_state,rt_country,qtd_state,qtd_country,norm_country,norm_rt_country,norm_qtd_country,acc_age
0,1495977545967317000,1481149613373538309,MattisseComedy,Tue Feb 22 04:23:35 +0000 2022,en,NaN,Up & Coming Welsh Comedian who isn't afraid t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
1,1495977557505753089,2821163219,Billington_Book,Tue Feb 22 04:23:38 +0000 2022,en,"Reus, Tarragonna, Spain","Disabled Vietnam vet, dad, granddad, cancer su...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2691
2,1495977563197476865,294162188,ArsenicMarko,Tue Feb 22 04:23:39 +0000 2022,en,"Aurora, Ontario",NaN,NaN,NaN,NaN,...,NaN,Canada,NaN,NaN,NaN,NaN,Canada,NaN,NaN,3944
3,1495977582155685895,1481149613373538309,MattisseComedy,Tue Feb 22 04:23:44 +0000 2022,en,NaN,Up & Coming Welsh Comedian who isn't afraid t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
4,1495977589151830017,1378410853314203649,Denise07190935,Tue Feb 22 04:23:45 +0000 2022,nl,NaN,🧠zelfnadenken \n🕊free Humanity \n📚boeken \nexi...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302697,1509017366180634626,1331144263657287680,Marek82085289,Wed Mar 30 03:59:10 +0000 2022,en,NaN,PO = KORUPCJA i ZDRAJCY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,490
1302698,1509017394299260931,1045050364158853120,life_canbeshort,Wed Mar 30 03:59:17 +0000 2022,en,Canada,The Universe Corrects All Wrongs✌️ pls dont sp...,NaN,NaN,NaN,...,NaN,Canada,NaN,NaN,NaN,NaN,Canada,NaN,NaN,1280
1302699,1509017428831145984,53870131,gusbayu,Wed Mar 30 03:59:25 +0000 2022,in,dipermukaan bumi,Ragadi musuh mepareng \nRihati ya tongwanya ta...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4650
1302700,1509017469758832640,777674879701254144,Online_SergioJu,Wed Mar 30 03:59:35 +0000 2022,es,"Santiago, Chile",Saludos ! aquí verás él Mundo a través de mis...,NaN,NaN,NaN,...,NaN,Chile,NaN,NaN,NaN,NaN,Chile,NaN,NaN,2018


In [7]:
twitter_dataset.columns

Index(['tweetid', 'userid', 'screen_name', 'date', 'lang', 'location',
       'description', 'place_id', 'place_url', 'place_type', 'place_name',
       'place_full_name', 'place_country_code', 'place_country',
       'place_bounding_box', 'text', 'extended', 'coord', 'reply_userid',
       'reply_screen', 'reply_statusid', 'tweet_type', 'friends_count',
       'listed_count', 'followers_count', 'favourites_count', 'statuses_count',
       'verified', 'hashtag', 'urls_list', 'profile_pic_url',
       'profile_banner_url', 'display_name', 'date_first_tweet',
       'account_creation_date', 'rt_urls_list', 'mentionid', 'mentionsn',
       'rt_screen', 'rt_userid', 'rt_user_description', 'rt_text',
       'rt_hashtag', 'rt_qtd_count', 'rt_rt_count', 'rt_reply_count',
       'rt_fav_count', 'rt_tweetid', 'rt_location', 'qtd_screen', 'qtd_userid',
       'qtd_user_description', 'qtd_text', 'qtd_hashtag', 'qtd_qtd_count',
       'qtd_rt_count', 'qtd_reply_count', 'qtd_fav_count', 'qtd_tweeti

In [43]:
#Testing

In [ ]:
list(twitter_dataset['tweet_type'])

In [ ]:
for idx in twitter_dataset.index:
    print(twitter_dataset['tweet_type'][idx],twitter_dataset['rt_urls_list'][idx])

In [50]:
#Checking date format given in dataset

In [8]:
print('Date format:',twitter_dataset['date'][0].strip())
month_dict = {}
for idx in twitter_dataset.index:
    temp = twitter_dataset['date'][idx].split(' ')[1]
    if temp not in month_dict:
        month_dict[temp] = 1
print(month_dict)

Date format: Tue Feb 22 04:23:35 +0000 2022
{'Feb': 1, 'Mar': 1, 'Apr': 1}


In [53]:
#Input date from user

In [3]:
month_query,day_query,year_query = takeDateQuery()

Input date in MM/DD/YYYY format02/23/2022
Feb 23 2022


In [54]:
#Compiling urls tweet/retweet from dataset

In [30]:
tweetid_urldata = []
urllist_urldata = []
for idx in twitter_dataset.index:
    tweetid_urldata.append(twitter_dataset['tweetid'][idx])
    if twitter_dataset['tweet_type'][idx] == 'retweeted_tweet_without_comment':
        urllist_urldata.append(expanded_url_from(twitter_dataset['rt_urls_list'][idx]))
    else:
        urllist_urldata.append(expanded_url_from(twitter_dataset['urls_list'][idx]))

In [55]:
#Confirming length of url list and dataset size

In [31]:
len(urllist_urldata)

1302702

In [34]:
#Extracting attributes, url ids and classifying url types from twitter dataset on the requested date

In [42]:
tweetid_list = []
userid_list = []
screenname_list = []
date_list = []
url_list = []
url_id_list = []
url_type_list = []
count_ids = 0
count_cids = 0
count_nonids = 0
#tweetid, userid, screen_name, date, url, url_id, url_type
for idx in twitter_dataset.index:
    date_from_dataset = twitter_dataset['date'][idx].split(' ')
    month_from_dataset = date_from_dataset[1]
    day_from_dataset = date_from_dataset[2]
    year_from_dataset = date_from_dataset[5]
    if month_from_dataset == month_query and day_from_dataset == day_query and year_from_dataset == year_query:
        tweet_id = twitter_dataset['tweetid'][idx]
        if tweet_id in tweetid_urldata:
            tweetid_index = tweetid_urldata.index(tweet_id)
            yt_url = urllist_urldata[tweetid_index]
            tweetid_list.append(tweet_id)
            userid_list.append(twitter_dataset['userid'][idx])
            screenname_list.append(twitter_dataset['screen_name'][idx])
            date_list.append(twitter_dataset['date'][idx])
            url_list.append(yt_url)
            link_type, link_id = getIDfromURL(yt_url)
            url_type_list.append(link_type)
            url_id_list.append(link_id)
            if link_type == 'other':
                count_nonids = count_nonids+1
            elif link_type == 'channel':
                count_cids = count_cids+1
            else:
                count_ids = count_ids+1
dated_twitterset = pd.DataFrame({'tweetid': tweetid_list, 
                                 'userid': userid_list, 
                                 'screen_name': screenname_list, 
                                 'date': date_list,
                                 'url_id': url_id_list,
                                 'url':url_list, 
                                 'url_type':url_type_list })
dated_twitterset

,tweetid,userid,screen_name,date,url_id,url,url_type
0,1496273610553634828,1470807188750815232,Notd3adTwist,Wed Feb 23 00:00:02 +0000 2022,-OD1A6hGMrA,https://youtu.be/-OD1A6hGMrA,video
1,1496273624428429316,19393766,jjj5819,Wed Feb 23 00:00:06 +0000 2022,wqvgy4rftJQ,https://youtu.be/wqvgy4rftJQ,video
2,1496273766086762497,1046421541108502528,billcreamer3,Wed Feb 23 00:00:39 +0000 2022,UI678QCZ1kU,https://youtu.be/UI678QCZ1kU,video
3,1496273825637490689,66918824,hiro_5,Wed Feb 23 00:00:54 +0000 2022,qUgzqkCW6A4,https://youtu.be/qUgzqkCW6A4,video
4,1496273835154415618,1188233964919689216,Kamil36850216,Wed Feb 23 00:00:56 +0000 2022,3OpG-6wAJhc,https://www.youtube.com/watch?v=3OpG-6wAJhc,video
...,...,...,...,...,...,...,...
18010,1496635913530716163,601798657,martinhowrylak,Wed Feb 23 23:59:42 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18011,1496635930744172555,3294054404,ewton_mallory,Wed Feb 23 23:59:46 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18012,1496635932866490371,1203922897238216705,mikrokupu,Wed Feb 23 23:59:47 +0000 2022,vFFhejGOTiM,https://youtu.be/vFFhejGOTiM,video
18013,1496635949668835334,936702533342584833,JsonBernardo,Wed Feb 23 23:59:51 +0000 2022,51Hz51cpFwQ,https://www.youtube.com/watch?v=51Hz51cpFwQ,video


In [56]:
#Verifying if all possible ids are extracted from the date queried dataset

In [31]:
count_cids,count_ids,count_nonids = countUrlType(dated_twitterset)
print(count_cids,count_ids,count_nonids)
checkSuccess(count_cids,count_ids,count_nonids, len(dated_twitterset))

118 17821 76


True

In [47]:
dated_twitterset.to_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}TW_YTdataset.csv',index=False)

In [4]:
month_query,day_query,year_query = takeDateQuery()

Input date in MM/DD/YYYY format02/23/2022
Feb 23 2022


In [5]:
dated_twitterset = pd.read_csv(f'./datasets/Extracted/{month_query}-{day_query}-{year_query}TW_YTdataset.csv')

In [6]:
dated_twitterset

,tweetid,userid,screen_name,date,url_id,url,url_type
0,1496273610553634828,1470807188750815232,Notd3adTwist,Wed Feb 23 00:00:02 +0000 2022,-OD1A6hGMrA,https://youtu.be/-OD1A6hGMrA,video
1,1496273624428429316,19393766,jjj5819,Wed Feb 23 00:00:06 +0000 2022,wqvgy4rftJQ,https://youtu.be/wqvgy4rftJQ,video
2,1496273766086762497,1046421541108502528,billcreamer3,Wed Feb 23 00:00:39 +0000 2022,UI678QCZ1kU,https://youtu.be/UI678QCZ1kU,video
3,1496273825637490689,66918824,hiro_5,Wed Feb 23 00:00:54 +0000 2022,qUgzqkCW6A4,https://youtu.be/qUgzqkCW6A4,video
4,1496273835154415618,1188233964919689216,Kamil36850216,Wed Feb 23 00:00:56 +0000 2022,3OpG-6wAJhc,https://www.youtube.com/watch?v=3OpG-6wAJhc,video
...,...,...,...,...,...,...,...
18010,1496635913530716163,601798657,martinhowrylak,Wed Feb 23 23:59:42 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18011,1496635930744172555,3294054404,ewton_mallory,Wed Feb 23 23:59:46 +0000 2022,XZGrP9siXwU,https://www.youtube.com/watch?v=XZGrP9siXwU,video
18012,1496635932866490371,1203922897238216705,mikrokupu,Wed Feb 23 23:59:47 +0000 2022,vFFhejGOTiM,https://youtu.be/vFFhejGOTiM,video
18013,1496635949668835334,936702533342584833,JsonBernardo,Wed Feb 23 23:59:51 +0000 2022,51Hz51cpFwQ,https://www.youtube.com/watch?v=51Hz51cpFwQ,video


In [7]:
#Segregating different types of IDs for channel, videos and other

In [20]:
video_id_list = []
channel_id_list = []
other_id_list = []
for idx in dated_twitterset.index:
    if dated_twitterset['url_type'][idx] == 'channel':
        channel_id_list.append(dated_twitterset['url_id'][idx])
    elif dated_twitterset['url_type'][idx] == 'video':
        video_id_list.append(dated_twitterset['url_id'][idx])
    else:
        other_id_list.append(dated_twitterset['url_id'][idx])    

In [21]:
checkSuccess(len(channel_id_list),len(video_id_list),len(other_id_list), len(dated_twitterset))

True

In [34]:
#Gathering unique video ids from the list

In [44]:
videoid_set = set(video_id_list)
video_id_list = list(videoid_set)
channelid_set = set(channel_id_list)
channel_id_list = list(channelid_set)

In [26]:
#Generating comma separated list of video ids to curb YouTube Data API calls

In [56]:
nums_video = len(video_id_list)
vid_req_num = int(nums_video/50)
videoid_index = 0
cs_videoids = ''
vid_req_list = []
for ind in range(0,vid_req_num):
    for s_ind in range(videoid_index,videoid_index+50):
        cs_videoids = cs_videoids+','+video_id_list[s_ind]
    vid_req_list.append(cs_videoids[1:])
    cs_videoids = ''
    videoid_index = videoid_index+50
if nums_video % 50 != 0:
    diff = nums_video - (50*vid_req_num)
    for s_ind in range(videoid_index,videoid_index+diff):
        cs_videoids = cs_videoids+','+video_id_list[s_ind]
    vid_req_list.append(cs_videoids[1:])
    cs_videoids = ''
    videoid_index = videoid_index+diff
print(f'No. of unique videos {videoid_index}, no of API calls needed: {len(vid_req_list)}')    

No. of unique videos 6459, no of API calls needed: 130


In [30]:
#Generating comma separated list of channel ids to curb YouTube Data API calls

In [57]:
nums_channel = len(channel_id_list)
ch_req_num = int(nums_channel/50)
channelid_index = 0
cs_channelids = ''
cid_req_list = []
for ind in range(0,ch_req_num):
    for s_ind in range(channelid_index,channelid_index+50):
        cs_channelids = cs_channelids+','+channel_id_list[s_ind]
    cid_req_list.append(cs_channelids[1:])
    cs_channelids = ''
    channelid_index = channelid_index+50
if nums_channel % 50 != 0:
    diff = nums_channel - (50*ch_req_num)
    for s_ind in range(channelid_index,channelid_index+diff):
        cs_channelids = cs_channelids+','+channel_id_list[s_ind]
    cid_req_list.append(cs_channelids[1:])
    cs_channelids = ''
    channelid_index = channelid_index+diff
print(f'No of channels {channelid_index}, no of API calls needed: {len(cid_req_list)}')

No of channels 31, no of API calls needed: 1


In [47]:
#Making API calls for fetching video metadata

In [59]:
response_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/videos?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={vid_req_list[0]}&key={YT_API_KEY}')
json_data = json.loads(response_from.text)
if 'items' in json_data:
    for x in range(1,len(vid_req_list)):
        response2_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/videos?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={vid_req_list[x]}&key={YT_API_KEY}')
        json_data_2 = json.loads(response2_from.text)
        if 'items' in json_data_2:
            json_data['items'] = [*json_data['items'],*json_data_2['items']]
        print(f'{x}/{len(vid_req_list)} processed',end="\r")
    print(len(json_data['items']))
else:
    print('some error occured fetching data from API')
    print(json_data)

5747130 processed


In [60]:
response_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/channels?part=snippet%2CcontentDetails%2Cstatistics&id={cid_req_list[0]}&key={YT_API_KEY}')
channel_json_data = json.loads(response_from.text)
if 'items' in channel_json_data:
    for x in range(1,len(cid_req_list)):
        response2_from = requests.get(f'https://youtube.googleapis.com/youtube/v3/channels?part=status%2Csnippet%2CcontentDetails%2Cstatistics%2CtopicDetails&id={cid_req_list[x]}&key={YT_API_KEY}')
        channel_json_data_2 = json.loads(response2_from.text)
        if 'items' in channel_json_data_2:
            channel_json_data['items'] = [*channel_json_data['items'],*channel_json_data_2['items']]
        print(f'{x}/{len(cid_req_list)} processed',end="\r")
    print(len(channel_json_data['items']))
else:
    print('some error occured fetching data from API')
    print(channel_json_data)

15
